<div class='bar_title'></div>

*Introduction to Data Science*

# Combining Data and Tidy Data

Gunther Gust<br>
Chair for Enterprise AI

Winter Semester 24/25

<img src='https://github.com/GuntherGust/tds2_data/blob/main/images/d3.png?raw=true' style='width:20%; float:left;' /> <br>



<img src='https://github.com/GuntherGust/tds2_data/blob/main/images/CAIDASlogo.png?raw=true' style='width:20%; float:left;' />

<img src="https://raw.githubusercontent.com/vhaus63/ids_data/main/ao_comb_tidy_data.png" style="width:80%; float:center;" />

# Last Lecture: Working with Data in Python

### What do you recall?

### Let's do a Mentimenter!

# Agenda for today

- ## Combining Data

    - ### Merging DataFrames
    - ### Joining DataFrames
    - ### Concatenating DataFrames

- ## Tidy Data
- ## Real-world Case Study



# Combining Data

There are 3 main ways of combining DataFrames together: **Merging**, **Joining** and **Concatenating**. In this lecture we will discuss these 3 methods with examples.

## Merging


There are different types of merges you can perform: inner, outer, left and right. The following picture describes what each method means.

<img src='https://raw.githubusercontent.com/vhaus63/ids_data/main/join_options.png' style='width:50%; float:left;' />

The `pd.merge` function implements a number of types of joins: one-to-one, many-to-one, and many-to-many. All three types of joins are accessed via an identical call to the `pd.merge` interface; the type of join performed depends on the form of the input data. We'll start with some simple examples of the three types of merges, and discuss detailed options a bit later.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Tim'],
                    'group': ['Accounting', 'Engineering',
                              'Engineering', 'HR', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue', 'Anne'],
                    'hire_date': [2004, 2008, 2012, 2014, 2006]})
display(df1, df2)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Tim,HR


,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014
4,Anne,2006


### Inner join

In [3]:
df3 = pd.merge(df1, df2, on= "employee")
display(df1, df2, df3) 

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Tim,HR


,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014
4,Anne,2006


,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


If the column name is identical in both original dataframes, you don't need to pass the `on=` argument:

In [4]:
df3b = pd.merge(df1, df2)
df3b

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### Left join

In [5]:
df3c = pd.merge(df1, df2, how= "left")
df3c

,employee,group,hire_date
0,Bob,Accounting,2008.0
1,Jake,Engineering,2012.0
2,Lisa,Engineering,2004.0
3,Sue,HR,2014.0
4,Tim,HR,NaN


### Right join

In [6]:
df3d = pd.merge(df1, df2, how= "right")
df3d

,employee,group,hire_date
0,Lisa,Engineering,2004
1,Bob,Accounting,2008
2,Jake,Engineering,2012
3,Sue,HR,2014
4,Anne,NaN,2006


### (Full) outer join

In [7]:
df3e = pd.merge(df1, df2, how= "outer")
df3e

,employee,group,hire_date
0,Anne,NaN,2006.0
1,Bob,Accounting,2008.0
2,Jake,Engineering,2012.0
3,Lisa,Engineering,2004.0
4,Sue,HR,2014.0
5,Tim,HR,NaN


The result of the merge is a new DataFrame that combines the information from the two inputs. Notice that the order of entries in each column is not necessarily maintained: in this case, the order of the employee column differs between df1 and df2, and the pd.merge function correctly accounts for this. Additionally, keep in mind that the merge in general discards the index, except in the special case of merges by index (see the left_index and right_index keywords).

### One-to-Many Joins

__One to one join:__

<img src='https://raw.githubusercontent.com/vhaus63/ids_data/main/inner_join_one_to_one.png' style='width:80%; float:left;' />

__One to many join__:

<img src='https://raw.githubusercontent.com/vhaus63/ids_data/main/inner_join_one_to_many.png' style='width:80%; float:left;' />


One-to-Many joins are joins in which one of the two key columns contains duplicate entries. For the One-to-Many case, the resulting DataFrame will preserve those duplicate entries as appropriate. Consider the following example of a One-to-Many join:


In [8]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display(df3, df4, pd.merge(df3, df4))

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


### Many-to-Many Joins


Many-to-many joins may be a bit confusing conceptually, but are nevertheless well defined. If the key column in both the left and right arrays contains duplicates, then the result is a many-to-many merge. This will be perhaps most clear with a concrete example. Consider the following, where we have a DataFrame showing one or more skills associated with a particular group. By performing a many-to-many join, we can recover the skills associated with any individual person:

In [9]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'software', 'math',
                               'spreadsheets', 'organization']})
display(df1, df5, pd.merge(df1, df5))

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Tim,HR


,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,software
3,Engineering,math
4,HR,spreadsheets
5,HR,organization


,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,software
3,Jake,Engineering,math
4,Lisa,Engineering,software
5,Lisa,Engineering,math
6,Sue,HR,spreadsheets
7,Sue,HR,organization
8,Tim,HR,spreadsheets
9,Tim,HR,organization


### Specification of the Merge Key

In [10]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display(df1, df3) 

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Tim,HR


,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [11]:
pd.merge(df1, df3, left_on="employee", right_on="name")

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


In [12]:
#get rid of extra column
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


## Joining

For convenience, Pandas includes the `DataFrame.join()` method, which performs an index-based merge without extra keywords. This is mostly a simplification of the already discussed `merge` method.

In [13]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display(df1a, df2a)

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
Tim,HR


,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014
Anne,2006


In [14]:
df1a.join(df2a)

# instead of: pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008.0
Jake,Engineering,2012.0
Lisa,Engineering,2004.0
Sue,HR,2014.0
Tim,HR,NaN


In [15]:
df1a.join(df2a, how='inner')

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


## Concatenation

Concatenation is all about glueing DataFrames together. Just remember that the dimensions need to match along the axis you’re concatenating on. You can use `pd.concat` and provide a list of DataFrames or Series objects to concatenate them:

In [16]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                        index=[0, 1, 2, 3])

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                        'B': ['B4', 'B5', 'B6', 'B7'],
                        'C': ['C4', 'C5', 'C6', 'C7'],
                        'D': ['D4', 'D5', 'D6', 'D7']},
                         index=[4, 5, 6, 7]) 

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                        'B': ['B8', 'B9', 'B10', 'B11'],
                        'C': ['C8', 'C9', 'C10', 'C11'],
                        'D': ['D8', 'D9', 'D10', 'D11']},
                        index=[8, 9, 10, 11])

display(df1, df2, df3)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


,A,B,C,D
8,A8,B8,C8,D8
9,A9,B9,C9,D9
10,A10,B10,C10,D10
11,A11,B11,C11,D11


### Along the Rows

<img src='https://raw.githubusercontent.com/vhaus63/ids_data/main/concat_axis0.avif' style='width:70%; float:left;' />

In [17]:
pd.concat([df1,df2,df3]) # default: axis=0

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9



### Along the Columns

<img src='https://raw.githubusercontent.com/vhaus63/ids_data/main/concat_axis1.avif' style='width:70%; float:left;' />

In [23]:
pd.concat([df1,df2,df3],axis=1)




,A,B,C,D,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,A4,B4,C4,D4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,A5,B5,C5,D5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,A6,B6,C6,D6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,A7,B7,C7,D7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A8,B8,C8,D8
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A9,B9,C9,D9


In [22]:
tmp = pd.merge(df1, df2, how= "outer")
full = pd.merge(tmp, df3, how= "outer")
full

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A10,B10,C10,D10
3,A11,B11,C11,D11
4,A2,B2,C2,D2
5,A3,B3,C3,D3
6,A4,B4,C4,D4
7,A5,B5,C5,D5
8,A6,B6,C6,D6
9,A7,B7,C7,D7


In [29]:
df1b = df1.set_index('A').add_suffix('_df1')
df2b = df2.set_index('A').add_suffix('_df2')    
df3b = df3.set_index('A').add_suffix('_df3')
df1c = df1b.join(df2b, how= "left")
df1c = df1c.join(df3b, how= "left")
df1c

,B_df1,C_df1,D_df1,B_df2,C_df2,D_df2,B_df3,C_df3,D_df3
A,,,,,,,,,
A0,B0,C0,D0,NaN,NaN,NaN,NaN,NaN,NaN
A1,B1,C1,D1,NaN,NaN,NaN,NaN,NaN,NaN
A2,B2,C2,D2,NaN,NaN,NaN,NaN,NaN,NaN
A3,B3,C3,D3,NaN,NaN,NaN,NaN,NaN,NaN


If the **indexes** of the dataframes are **identical**, no `NaN` values are created. (But with overlapping indices, using `pd.merge()` / `join()` is preferable.)

 ### Exercise 1

 The following tables are given:

In [30]:
employees = pd.DataFrame({
    'employee_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Carol', 'David', 'Eve'],
    'department_id': [101, 102, 103, 101, None]
})

departments = pd.DataFrame({
    'department_id': [101, 102, 103, 104],
    'department_name': ['Sales', 'Engineering', 'Marketing', 'HR']
})

projects = pd.DataFrame({
    'project_id': ['P1', 'P2', 'P3', 'P4', 'P6'],
    'employee_id': [1, 1, 2, 3, 6],
    'project_name': ['Market Analysis', 'Sales Strategy', 'Product Design', 'Campaign', 'Special Project']
})

display(employees, departments, projects)

,employee_id,name,department_id
0,1,Alice,101.0
1,2,Bob,102.0
2,3,Carol,103.0
3,4,David,101.0
4,5,Eve,NaN


,department_id,department_name
0,101,Sales
1,102,Engineering
2,103,Marketing
3,104,HR


,project_id,employee_id,project_name
0,P1,1,Market Analysis
1,P2,1,Sales Strategy
2,P3,2,Product Design
3,P4,3,Campaign
4,P6,6,Special Project


What kind of commands need to be performed in order to get the following result? Is it a one to one or a one to many or a many to many join? Recreate the table.



 <img src='https://raw.githubusercontent.com/vhaus63/ids_data/main/merged_table_lecture_exercise.png' style='width:70%; float:left;' />

In [33]:
# Your code here...

nf = pd.merge(employees, departments, on= "department_id", how= "left")
pd.merge(nf, projects, on= "employee_id", how= "left")

,employee_id,name,department_id,department_name,project_id,project_name
0,1,Alice,101.0,Sales,P1,Market Analysis
1,1,Alice,101.0,Sales,P2,Sales Strategy
2,2,Bob,102.0,Engineering,P3,Product Design
3,3,Carol,103.0,Marketing,P4,Campaign
4,4,David,101.0,Sales,NaN,NaN
5,5,Eve,NaN,NaN,NaN,NaN


# Tidy data

We will learn a consistent way to organise data in Python, an organisation called __tidy data__. Getting your data into this format requires some upfront work, but that work pays off in the long term. Once you have tidy data and pandas data frames, you will spend __much less time__ munging data from one representation to another, allowing you to spend more time on the analytic questions at hand.

This chapter will give you a practical introduction to tidy data and the accompanying tools in the `pandas` package.

## Example

You can represent the same underlying data in multiple ways. The example below shows the same data organised in __four different ways__. Each dataset shows the same values of four variables *country*, *year*, *population*, and *cases*, but each dataset organises the values in a different way.


In [34]:
base_url = "https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/"
table1 = pd.read_csv("{}table1.csv".format(base_url))
table2 = pd.read_csv("{}table2.csv".format(base_url))
table3 = pd.read_csv("{}table3.csv".format(base_url))
table4a = pd.read_csv("{}table4a.csv".format(base_url))
table4b = pd.read_csv("{}table4b.csv".format(base_url))
table5 = pd.read_csv("{}table5.csv".format(base_url), dtype = 'object')

 ### 1. Cases and population in separate columns

In [35]:
display(table1)

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


### 2. Type of measurement in one column, count in the other

In [36]:
display(table2)

,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272


### 3. rate=cases/population in one column

In [37]:
display(table3) 

,country,year,rate
0,Afghanistan,1999,745/19987071
1,Afghanistan,2000,2666/20595360
2,Brazil,1999,37737/172006362
3,Brazil,2000,80488/174504898
4,China,1999,212258/1272915272
5,China,2000,213766/1280428583


### 4. Spread across two DataFrames

In [38]:
display(table4a)  # cases
display(table4b)  # population

,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


,country,1999,2000
0,Afghanistan,19987071,20595360
1,Brazil,172006362,174504898
2,China,1272915272,1280428583


### Motivating example

Compute the `rate` for `table1`, `table2`, and `table4a` + `table4b`.
    You will need to perform four operations:
1. Extract the number of TB cases per country per year.
2. Extract the matching population per country per year.
3. Divide cases by population
4. Store back in the appropriate place.

#### Which representation is easiest to work with? Which is hardest? Why?

These are all representations of the same underlying data, but they are not equally easy to use. One dataset, the tidy dataset, will be much easier to work with.



## Characteristics of Tidy Data


There are three __interrelated rules__ which make a dataset tidy:

1.  Each __variable__ must have its own __column.__

2.  Each __observation__ must have its own __row.__

3.  Each __value__ must have its own __cell.__


Why ensure that your data is tidy? There are two __main advantages:__

1.  There's a general advantage to picking one __consistent way of storing data.__ If you have a consistent data structure, it's easier to learn the tools that work with it because they have an underlying __uniformity.__

2.  There's a specific advantage to placing variables in columns because it allows __Pandas' and NumPy's vectorised nature to shine.__

The principles of tidy data seem so obvious that you might wonder if you'll ever encounter a dataset that isn't tidy. Unfortunately, however, most data that you will encounter will be untidy. There are two main reasons:

1.  Most __people aren't familiar__ with the principles of tidy data, and it's hard to derive them yourself unless you spend a _lot_ of time working with data.

1.  Data is often organised to facilitate some __use other than analysis.__ For  example, data is often organised to make __entry__ as easy as possible.


This means for most real analyses, you'll need to do some tidying. 

1. __The first step__ is always to figure out what the __variables and observations__ are. Sometimes this is easy; other times you'll need to consult with the people who originally generated the data.

2. The second step is to __resolve one of two common problems:__

    1. One variable might be spread across multiple columns.

    1. One observation might be scattered across multiple rows.




Typically a dataset will only suffer from one of these problems; To fix these problems, you'll need three functions in pandas: `melt()`, `pivot()`, and `pivot_table()`. There are two additional functions called `stack()` and `unstack()` that use multi-index columns and rows. Pandas provides a guide to [reshaping and pivot tables](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#reshaping) in their user guide.



### Longer (`melt()`)

A common problem is a dataset where some of the column names are not names of variables, but _values_ of a variable. Take `table4a`: the column names `1999` and `2000` represent values of the `year` variable, the values in the `1999` and `2000` columns represent values of the `cases` variable, and each row represents two observations, not one.

In [39]:
table4a

,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


To tidy a dataset like this, we need to __stack__ the offending columns into a new pair of variables. To describe that operation we need three parameters:

* The set of columns whose names are identifier variables, not values. In this example,
  `country` is the identifier column and the value columns are `1999` and `2000`.



* The name of the variable to move the column names to. Here it is `year`.


* The name of the variable to move the column values to. Here it's `cases`.


Together those parameters generate the call to `melt()`:

In [40]:
table4a.melt(['country'], var_name = "year", value_name = "cases")

,country,year,cases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
3,Afghanistan,2000,2666
4,Brazil,2000,80488
5,China,2000,213766


`year` and `cases` do not exist in `table4a` so we put their names in quotes.

In the final result, the pivoted columns are dropped, and we get new `year` and `cases` columns. Otherwise, the relationships between the original variables are preserved. Visually, this is shown in Figure \@ref(fig:tidy-gather).

`melt()` makes datasets longer (imagine melting icecream down the cone) by increasing the number of rows and decreasing the number of columns. I don’t believe it makes sense to describe a dataset as being in “long form”. Length is a relative term, and you can only say that dataset A is longer than dataset B.



We can use `melt()` to tidy `table4b` in a similar fashion. The only difference is the variable stored in the cell values:

In [41]:
table4b.melt(['country'], var_name = 'year', value_name = 'population')

,country,year,population
0,Afghanistan,1999,19987071
1,Brazil,1999,172006362
2,China,1999,1272915272
3,Afghanistan,2000,20595360
4,Brazil,2000,174504898
5,China,2000,1280428583


To combine the tidied versions of `table4a` and `table4b` into a single DataFrame, we need to use `merge()`.

In [42]:
tidy4a = table4a.melt(['country'], var_name = "year", value_name = "cases")
tidy4b = table4b.melt(['country'], var_name = 'year', value_name = 'population')
pd.merge(tidy4a, tidy4b, on = ['country', 'year'])

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Brazil,1999,37737,172006362
2,China,1999,212258,1272915272
3,Afghanistan,2000,2666,20595360
4,Brazil,2000,80488,174504898
5,China,2000,213766,1280428583


### Wider (`pivot()`)

`pivot()` is the opposite of `melt()`. You use it when an observation is scattered across multiple rows. For example, take `table2`: an observation is a country in a year, but each observation is spread across two rows.


In [43]:
table2

,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272


To tidy this up, we first analyse the representation in similar way to `melt()`. This time, however, we only need two parameters:

* The column to take variable names from. Here, it's `type`.

* The column to take values from. Here it's `count`.



Once we've figured that out, we can use `pivot()`.

In this example, we have a multi-column `index` argument and will need to use `pivot_table()`.  With a single column index `pivot()` can be used.

In [44]:
table2.pivot_table(
    index = ['country', 'year'], 
    columns = 'type', 
    values = 'count').reset_index()

type,country,year,cases,population
0,Afghanistan,1999,745.0,1.998707e+07
1,Afghanistan,2000,2666.0,2.059536e+07
2,Brazil,1999,37737.0,1.720064e+08
3,Brazil,2000,80488.0,1.745049e+08
4,China,1999,212258.0,1.272915e+09
5,China,2000,213766.0,1.280429e+09


As you might have guessed from their names, `pivot()` and `pivot_table()` are complements to `melt()`. `melt()` makes wide tables narrower and longer; `pivot()` and `pivot_table()` makes long tables shorter and wider.


 ### Exercise 2



 You’re given a DataFrame containing survey results from different years, with each year as a separate column. The data currently looks like this:

In [45]:
data = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Carol', 'David'],
    '2020': [3, 2, 5, None],
    '2021': [None, 3, 4, 4],
    '2022': [2, 5, None, 3]
})

data


,Name,2020,2021,2022
0,Alice,3.0,NaN,2.0
1,Bob,2.0,3.0,5.0
2,Carol,5.0,4.0,NaN
3,David,NaN,4.0,3.0


 Reshape the data such that it becomes tidy and fill any missing value in the value column with 0.

In [56]:
# Your code here...
data.isnull()
tidy_data = data.fillna(0)
tidy_data.melt(["Name"], var_name = "year", value_name = "value")

,Name,year,value
0,Alice,2020,3.0
1,Bob,2020,2.0
2,Carol,2020,5.0
3,David,2020,0.0
4,Alice,2021,0.0
5,Bob,2021,3.0
6,Carol,2021,4.0
7,David,2021,4.0
8,Alice,2022,2.0
9,Bob,2022,5.0


## Control Questions: Mentimeter

## Exercise: Real-world data tidying problem

To finish off the chapter, let's pull together everything you've learned to tackle a realistic data tidying problem. The `tidyr::who` dataset contains tuberculosis (TB) cases broken down by year, country, age, gender, and diagnosis method. The data comes from the [2014 World Health Organization Global Tuberculosis Report](http://www.who.int/tb/country/data/download/en/).

There's a wealth of epidemiological information in this dataset, but it's challenging to work with the data in the form that it's provided:

In [57]:
who = pd.read_csv("https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/who.csv")
# Fix the Namibia NA iso code.
who.loc[who.iso2.isna(),'iso2'] = "NA"

who

,country,iso2,iso3,year,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,new_sp_m5564,...,newrel_m4554,newrel_m5564,newrel_m65,newrel_f014,newrel_f1524,newrel_f2534,newrel_f3544,newrel_f4554,newrel_f5564,newrel_f65
0,Afghanistan,AF,AFG,1980,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AF,AFG,1981,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AF,AFG,1982,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AF,AFG,1983,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AF,AFG,1984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7235,Zimbabwe,ZW,ZWE,2009,125.0,578.0,NaN,3471.0,681.0,293.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7236,Zimbabwe,ZW,ZWE,2010,150.0,710.0,2208.0,1682.0,761.0,350.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7237,Zimbabwe,ZW,ZWE,2011,152.0,784.0,2467.0,2071.0,780.0,377.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7238,Zimbabwe,ZW,ZWE,2012,120.0,783.0,2421.0,2086.0,796.0,360.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This is a very typical real-life example dataset. It contains redundant columns, odd variable codes, and many missing values. In short, `who` is messy, and we'll need multiple steps to tidy it. 

This endeavor will be part of the following exercise.

<img src='https://raw.githubusercontent.com/vhaus63/ids_data/main/d3.png' style='width:80%; float:left;' />